In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import json
import cv2
import numpy as np

In [2]:
device = torch.device("cuda")
device

device(type='cuda')

Creating torch Dataset

In [3]:
class KeypointsData(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)

        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.458, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        def __len__(self):
            return len(self.data)
        
        def __getitem__(self, idx):
            item = self.data[idx]
            img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
            h, w = img.shape[:2]

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = self.transforms(img)
            kps = np.array(item['kps']).flatten()
            kps = kps.astype(np.float32)

            kps[::2] *= 224.0 / w
            kps[1::2] *= 224.0 / h

            return img, kps

In [ ]:
train_dataset = KeypointsData('data/images', 'data/data_train.json')
val_dataset = KeypointsData('data/images', 'data/data_val.json')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_laoder = DataLoader(val_dataset, batch_size=8, shuffle=True)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Pranav Darekar\AppData\Local\Temp\ipykernel_27776\1740564180.py:1: SyntaxWarning: invalid escape sequence '\d'
  train_dataset = KeypointsData('data\data_train.json')
C:\Users\Pranav Darekar\AppData\Local\Temp\ipykernel_27776\1740564180.py:1: SyntaxWarning: invalid escape sequence '\d'
  train_dataset = KeypointsData('data\data_train.json')


TypeError: KeypointsData.__init__() missing 1 required positional argument: 'data_file'

Model

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)

In [ ]:
model = model.to(device)

In [9]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs = 20
for epoch in range(epochs):
    for i, (imgs, kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()
        
        if i%10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")